In [3]:
import os
import pandas as pd
from glob import glob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from monai.transforms import (
    LoadImaged, EnsureChannelFirstd, ScaleIntensityd, ResizeWithPadOrCropd,
    ToTensord, Compose
)
from monai.networks import nets  # This replaces 'UnetEncoder'
from monai.networks.layers import Norm
from sklearn.model_selection import train_test_split
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [4]:
class TumorDataset(Dataset):
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        
        # Load the image
        image = nib.load(image_path).get_fdata()

        sample = {'image': image, 'label': label}

        if self.transforms:
            sample = self.transforms(sample)

        return sample

# Set up transforms
transforms = Compose([
    LoadImaged(keys=["image", "label"]),
    EnsureChannelFirstd(keys=["image", "label"]),
    ScaleIntensityd(keys=["image"]),
    ResizeWithPadOrCropd(keys=["image", "label"], spatial_size=(128, 128, 128)),
    ToTensord(keys=["image", "label"]),
])

# Prepare dataset
image_paths = glob('D:/Major_Project_(CSE)/Brain_Tumor_Detection/combined_dataset/images/*.nii.gz')
labels = pd.read_csv('D:/Major_Project_(CSE)/Brain_Tumor_Detection/combined_dataset/labels.csv')['label'].values

# Create Dataset and DataLoader
dataset = TumorDataset(image_paths=image_paths, labels=labels, transforms=transforms)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)

print(f"Number of samples: {len(dataset)}")


Number of samples: 739


In [5]:
from monai.networks.nets import UNet
from monai.networks.layers import Norm

# Define the 3D U-Net model
model = UNet(
    spatial_dims=3,
    in_channels=1,  # Single-channel (grayscale) MRI images
    out_channels=1,  # Tumor segmentation (binary: tumor vs no-tumor)
    channels=(16, 32, 64, 128, 256),  # Channels at each level of the U-Net
    kernel_size=3,
    strides=(2, 2, 2, 2),  # Strides at each level (matching the number of channels)
    norm=Norm.BATCH,
    act="relu",
)

# Move the model to the GPU
model = model.to(device)

# Print model summary
print(model)


UNet(
  (model): Sequential(
    (0): Convolution(
      (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
      (adn): ADN(
        (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (D): Dropout(p=0.0, inplace=False)
        (A): ReLU()
      )
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): Convolution(
          (conv): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): ReLU()
          )
        )
        (1): SkipConnection(
          (submodule): Sequential(
            (0): Convolution(
              (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
              (adn): ADN(
                (N): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=

In [26]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split

from monai.transforms import (
    Compose, LoadImaged, EnsureChannelFirstd, ScaleIntensityd,
    Resized, ToTensord
)
from monai.data import CacheDataset
from monai.networks.nets import UNet

# ------------------ Paths ------------------
images_dir = "D:/Major_Project_(CSE)/Brain_Tumor_Detection/combined_dataset/images"
labels_csv = "D:/Major_Project_(CSE)/Brain_Tumor_Detection/combined_dataset/labels.csv"

# ------------------ Load labels ------------------
df = pd.read_csv(labels_csv)
if "filename" not in df.columns or "label" not in df.columns:
    raise ValueError("❌ CSV must contain 'filename' and 'label' columns!")

# 🧠 Instead of scalar labels, we'll turn them into full-volume masks
def generate_fake_mask(label):
    # Returns a 5D tensor: (1, 1, 128, 128, 128) instead of (1, 128, 128, 128)
    return torch.ones((1, 1, 128, 128, 128)) if label == 1 else torch.zeros((1, 1, 128, 128, 128))

# Build dataset dicts
data_dicts = [
    {"image": os.path.join(images_dir, row["filename"]), "label": int(row["label"])}
    for _, row in df.iterrows()
]

train_files, val_files = train_test_split(data_dicts, test_size=0.2, random_state=42)

# ------------------ Transforms ------------------
common_transforms = Compose([
    LoadImaged(keys=["image"]),
    EnsureChannelFirstd(keys=["image"]),
    ScaleIntensityd(keys=["image"]),
    Resized(keys=["image"], spatial_size=(128, 128, 128)),
    ToTensord(keys=["image"]),
])

# Apply transform to label manually
class SegmentationDataset(torch.utils.data.Dataset):
    def __init__(self, data, transforms):
        self.data = data
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        d = self.transforms({"image": self.data[idx]["image"]})
        label_tensor = generate_fake_mask(self.data[idx]["label"])  # shape: (1, 1, 128, 128, 128)
        return {"image": d["image"], "label": label_tensor}

# ------------------ Dataset ------------------
train_ds = SegmentationDataset(train_files, common_transforms)
val_ds = SegmentationDataset(val_files, common_transforms)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=1)

# ------------------ Model ------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=1,  # Binary segmentation
    channels=(16, 32, 64, 128, 256),
    strides=(2, 2, 2, 2),
    num_res_units=2,
).to(device)

# ------------------ Training ------------------
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 20
print("🔧 Starting training...")
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for batch in train_loader:
        inputs = batch["image"].to(device)
        labels = batch["label"].to(device)

        # Squeeze the labels to match the shape of outputs
        labels = labels.squeeze(1)  # Convert from (1, 1, 128, 128, 128) to (1, 128, 128, 128)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"✅ Epoch {epoch+1} loss: {epoch_loss / len(train_loader):.4f}")

# ------------------ Save Model ------------------
torch.save(model.state_dict(), "3d_unet_brain_tumor_segmenter.pth")
print("✅ Saved trained segmentation model!")


🔧 Starting training...
✅ Epoch 1 loss: 0.6902
✅ Epoch 2 loss: 0.6419
✅ Epoch 3 loss: 0.5675
✅ Epoch 4 loss: 0.5577
✅ Epoch 5 loss: 0.4825
✅ Epoch 6 loss: 0.4466
✅ Epoch 7 loss: 0.4195
✅ Epoch 8 loss: 0.3975
✅ Epoch 9 loss: 0.3793
✅ Epoch 10 loss: 0.4244
✅ Epoch 11 loss: 0.3773
✅ Epoch 12 loss: 0.3424
✅ Epoch 13 loss: 0.3366
✅ Epoch 14 loss: 0.3182
✅ Epoch 15 loss: 0.3057
✅ Epoch 16 loss: 0.2951
✅ Epoch 17 loss: 0.2848
✅ Epoch 18 loss: 0.2748
✅ Epoch 19 loss: 0.2651
✅ Epoch 20 loss: 0.2560
✅ Saved trained segmentation model!


In [32]:
from monai.metrics import DiceMetric
from sklearn.metrics import jaccard_score, precision_score, recall_score, f1_score
import numpy as np
import torch

# Accuracy function
def compute_accuracy(pred_mask, true_mask):
    pred_flat = pred_mask.flatten()
    true_flat = true_mask.flatten()
    return np.mean(pred_flat == true_flat)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# Metrics containers
dice_metric = DiceMetric(include_background=True, reduction="mean")
dice_vals, iou_vals, precision_vals, recall_vals, f1_vals, acc_vals = [], [], [], [], [], []

with torch.no_grad():
    for batch in val_loader:
        inputs = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = model(inputs)                         # (B, 1, 128, 128, 128)
        outputs = torch.sigmoid(outputs)                # Apply sigmoid for binary prediction
        preds = (outputs > 0.5).float()                 # Threshold at 0.5

        # Dice (on GPU)
        dice_score = dice_metric(preds, labels)
        dice_vals.append(dice_score.item())

        # Move to CPU for scikit-learn metrics
        preds_np = preds.cpu().numpy().squeeze()
        labels_np = labels.cpu().numpy().squeeze()

        pred_flat = preds_np.flatten()
        label_flat = labels_np.flatten()

        iou_vals.append(jaccard_score(label_flat, pred_flat, zero_division=0))
        precision_vals.append(precision_score(label_flat, pred_flat, zero_division=0))
        recall_vals.append(recall_score(label_flat, pred_flat, zero_division=0))
        f1_vals.append(f1_score(label_flat, pred_flat, zero_division=0))
        acc_vals.append(compute_accuracy(preds_np, labels_np))

# Final output
print("\n🔍 Evaluation Results:")
print(f"🎯 Avg Dice:     {np.mean(dice_vals):.4f}")
print(f"📦 Avg IoU:      {np.mean(iou_vals):.4f}")
print(f"✅ Avg Precision:{np.mean(precision_vals):.4f}")
print(f"🔁 Avg Recall:   {np.mean(recall_vals):.4f}")
print(f"📊 Avg F1 Score: {np.mean(f1_vals):.4f}")
print(f"📈 Avg Accuracy: {np.mean(acc_vals):.4f}")



🔍 Evaluation Results:
🎯 Avg Dice:     nan
📦 Avg IoU:      0.5811
✅ Avg Precision:0.5811
🔁 Avg Recall:   0.5811
📊 Avg F1 Score: 0.5811
📈 Avg Accuracy: 0.9999


In [33]:
# Initialize manually to track Dice
dice_vals = []

with torch.no_grad():
    for batch in val_loader:
        inputs = batch["image"].to(device)
        labels = batch["label"].to(device)

        outputs = model(inputs)
        outputs = torch.sigmoid(outputs)
        preds = (outputs > 0.5).float()

        # Only compute Dice if either pred or label is not completely empty
        if torch.sum(labels) > 0 or torch.sum(preds) > 0:
            dice_score = (2. * (preds * labels).sum()) / (preds.sum() + labels.sum() + 1e-5)
            dice_vals.append(dice_score.item())
        else:
            dice_vals.append(1.0)  # Both empty = perfect match (optional)

        # Rest of your code...

# Then at the end:
print(f"🎯 Avg Dice:     {np.mean(dice_vals):.4f}")


🎯 Avg Dice:     0.5811


In [24]:
print(f"Labels shape: {labels.shape}")


Labels shape: torch.Size([1])
